<a href="https://colab.research.google.com/github/SinanHinislioglu/deploying-machine-learning-models/blob/master/pytorch_create_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')

In [4]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [5]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [6]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

In [10]:
Xtrain_

tensor([[-0.0935,  0.2215],
        [ 1.2217, -0.5342],
        [-1.0655,  0.4104],
        ...,
        [ 1.2789,  0.4104],
        [-0.9512,  0.2215],
        [-1.2943, -1.4316]])

In [11]:
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [12]:
ytrain_

tensor([1, 1, 1,  ..., 1, 1, 0])

In [13]:
Xtrain_.shape, ytrain_.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [14]:
Xtest_.shape, ytest_.shape

(torch.Size([311, 2]), torch.Size([311]))

In [15]:
input_size=2
output_size=2
hidden_size=10

In [16]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = torch.relu((self.fc1(X)))
       X = torch.relu((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1)

In [17]:
model = Net()



In [18]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [19]:
epochs = 100


In [20]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

Epoch 0 loss 0.7795737385749817
Epoch 1 loss 0.7501122951507568
Epoch 2 loss 0.7256433367729187
Epoch 3 loss 0.7054636478424072
Epoch 4 loss 0.6889753341674805
Epoch 5 loss 0.6743037104606628
Epoch 6 loss 0.6618015170097351
Epoch 7 loss 0.6502671837806702
Epoch 8 loss 0.6391473412513733
Epoch 9 loss 0.6283650398254395
Epoch 10 loss 0.6176949739456177
Epoch 11 loss 0.6068593263626099
Epoch 12 loss 0.595531702041626
Epoch 13 loss 0.5839495062828064
Epoch 14 loss 0.5721211433410645
Epoch 15 loss 0.559948205947876
Epoch 16 loss 0.5475040674209595
Epoch 17 loss 0.5346882939338684
Epoch 18 loss 0.5217557549476624
Epoch 19 loss 0.50855952501297
Epoch 20 loss 0.4952520728111267
Epoch 21 loss 0.4820522367954254
Epoch 22 loss 0.46874213218688965
Epoch 23 loss 0.45517387986183167
Epoch 24 loss 0.4414069950580597
Epoch 25 loss 0.4273139536380768
Epoch 26 loss 0.4129120111465454
Epoch 27 loss 0.3982292413711548
Epoch 28 loss 0.3832625150680542
Epoch 29 loss 0.36812275648117065
Epoch 30 loss 0.35276

In [21]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.9505, -0.5949],
         [ 0.0889,  0.0549],
         [ 0.7213,  0.7335],
         [ 0.8104, -0.3419],
         [ 1.2182,  0.2120],
         [ 0.0394,  0.3640],
         [-0.5602,  0.5198],
         [-0.8217, -0.7232],
         [ 0.4107,  0.7791],
         [-0.4839,  1.0974]], requires_grad=True),
 Parameter containing:
 tensor([ 0.5990, -0.4136,  0.8605,  0.2096,  0.2775,  1.1981,  0.1953, -0.0657,
          0.4408,  0.5192], requires_grad=True),
 Parameter containing:
 tensor([[-0.2655,  0.1077, -0.1921, -0.3390, -0.1297, -0.2182,  0.0390, -0.1467,
           0.1502,  0.0816],
         [ 0.6635, -0.0689, -0.2109, -0.3662,  0.0628,  0.2157, -0.0977,  0.4081,
          -0.1696, -0.5458],
         [-0.2114, -0.2633,  0.7494,  0.2966,  0.6288,  0.5969,  0.3171, -0.6209,
           0.4263,  0.9956],
         [-0.1158,  0.0352,  0.0057, -0.3148, -0.2934, -0.0320, -0.2694,  0.2076,
          -0.2135, -0.1945],
         [-0.2053,  0.2357, -0.2479, -0.1596,

In [22]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

tensor([[-0.2650, -1.7622]])

In [23]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0081, -4.8205]], grad_fn=<LogSoftmaxBackward0>)

In [24]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [25]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.0836, -0.4130]], grad_fn=<LogSoftmaxBackward0>)

In [26]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

In [27]:
torch.save(model,'customer_buy.pt')

In [28]:
!ls

customer_buy.pt  sample_data


In [29]:
restored_model = torch.load('customer_buy.pt')

In [30]:
y_cust_20_40000 = restored_model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0081, -4.8205]], grad_fn=<LogSoftmaxBackward0>)

In [31]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [32]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[-0.9505, -0.5949],
                      [ 0.0889,  0.0549],
                      [ 0.7213,  0.7335],
                      [ 0.8104, -0.3419],
                      [ 1.2182,  0.2120],
                      [ 0.0394,  0.3640],
                      [-0.5602,  0.5198],
                      [-0.8217, -0.7232],
                      [ 0.4107,  0.7791],
                      [-0.4839,  1.0974]])),
             ('fc1.bias',
              tensor([ 0.5990, -0.4136,  0.8605,  0.2096,  0.2775,  1.1981,  0.1953, -0.0657,
                       0.4408,  0.5192])),
             ('fc2.weight',
              tensor([[-0.2655,  0.1077, -0.1921, -0.3390, -0.1297, -0.2182,  0.0390, -0.1467,
                        0.1502,  0.0816],
                      [ 0.6635, -0.0689, -0.2109, -0.3662,  0.0628,  0.2157, -0.0977,  0.4081,
                       -0.1696, -0.5458],
                      [-0.2114, -0.2633,  0.7494,  0.2966,  0.6288,  0.5969,  0.3171

In [33]:
torch.save(model.state_dict(),'customer_buy_state_dict')

In [34]:
!ls

customer_buy.pt  customer_buy_state_dict  sample_data


In [35]:
new_predictor = Net()

In [36]:
y_cust_20_40000 = new_predictor(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.5395, -0.8747]], grad_fn=<LogSoftmaxBackward0>)

In [37]:
!zip -r customer_buy_state_dict.zip customer_buy_state_dict

  adding: customer_buy_state_dict (deflated 49%)


In [38]:
!ls

customer_buy.pt		 customer_buy_state_dict.zip
customer_buy_state_dict  sample_data


In [39]:
from google.colab import files


In [40]:
files.download('customer_buy_state_dict.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>